In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import cx_Oracle
from datetime import datetime

## 1. connect

oracle 中“模式(schema)”：oracle 在创建用户时会自动创建同名的模式，该用户的所有对象都会归属此模式。  
操作时会默认为该用户的模式，所以访问此模式下的对象时可不加。

In [3]:
user = 'jzsk'
passwd = 'jzsk'
host = '10.10.10.155'
port = '1521'
ins = 'ORCL'

table = 'JD_IMAGE_DETECT'
sequence = 'JD_IMAGE_DETECT_ID_SEQ'
directory = 'JD_IMAGE_PATH'
procedure = 'saveimg2disk'

In [4]:
db = cx_Oracle.connect(
    f'{user}/{passwd}@{host}:{port}/{ins}',
    encoding="UTF-8")
# db.current_schema = user  # 无须手动，默认即当前用户的模式

## 2. 添加数据表

JD_image_detect 表格:

| *ID* | DATA_CAMERA_ID | DATE_TIME | DATA_IMAGE_PATH | DATA_INFOR | DATA_BLOB |
| :--: | :--: | :--: | :--: | :--: | :--: |
| int |    int    |    date    |   varchar(255)     |  varchar(255)  | BLOB |
| 主键 | 探头ID    |    时间戳   |   图片保存的路径  |  图片信息 | 图片二进制数据 |

In [6]:
# table = 'JD_001'
sql = """CREATE TABLE {table} (
                ID INT PRIMARY KEY NOT NULL,
                DATA_CAMERA_ID INT NOT NULL,
                DATE_TIME DATE NOT NULL,
                DATA_IMAGE_PATH VARCHAR(255) NOT NULL,
                DATA_INFOR VARCHAR(255) NOT NULL,
                DATA_BLOB BLOB
                )"""
# sql_drop = """DROP TABLE {table}"""
with db.cursor() as cursor:
    cursor.execute(sql.format(table=table))


## 2.1 创建 sequence

In [7]:

sql = """CREATE SEQUENCE {sequence}
    MINVALUE 1
    NOMAXVALUE
    START WITH 1
    INCREMENT BY 1
    NOCACHE
"""
# sql_drop = """DROP SEQUENCE {sequence}"""
with db.cursor() as cursor:
    cursor.execute(sql.format(sequence=sequence))

## 2.2 创建触发器

**错误：此处提交的触发器不可用，需要在 navicat 中手动添加**

In [79]:
# sequence = 'JD_IMAGE_DETECT_ID_SEQ'
# trigger = 'JD_IMAGE_DETECT_ID_TRIGGER'
# table = 'JD_001'

# sql = """create or replace trigger {trigger}
#     before insert on {table}
#     for each row
#     begin
#         select {sequence}.nextval into :new.id from dual;
#     end
# """

# sql_drop = """drop trigger {trigger}"""
# with db.cursor() as cursor:
#     cursor.execute(sql.format(user=user, trigger=trigger, table=table, sequence=sequence))


## 3. 提交数据

### 3.1 获取下一个 id

In [73]:
sql = """select * from user_sequences where sequence_name='{sequence}'"""

with db.cursor() as cursor:
    cursor.execute(sql.format(sequence=sequence))
    if cursor.fetchone() is None:
        print(f'Sequence {sequence} not exist, try to create it.')

<cx_Oracle.Cursor on <cx_Oracle.Connection to jzsk@10.10.10.155:1521/ORCL>>

In [74]:
sql = """select table_name from user_tables where table_name='{table}'"""

with db.cursor() as cursor:
    cursor.execute(sql.format(table=table))
    if cursor.fetchone() is None:
        print(f'Table {table} not exist, try to create it.')

<cx_Oracle.Cursor on <cx_Oracle.Connection to jzsk@10.10.10.155:1521/ORCL>>

table JD_IMAGE_DETECT does not exist, try to create it.


In [ ]:
sql = """select * from dba_directories where directory_name='{directory}'"""

with db.cursor() as cursor:
    cursor.execute(sql.format(directory=directory))
    if cursor.fetchone() is None:
        print(f'Directory {directory} not exist, try to create it.')

In [84]:
sql = """select * from user_procedures where object_name='{procedure}'"""

with db.cursor() as cursor:
    cursor.execute(sql.format(procedure=procedure.upper()))
    if cursor.fetchone() is None:
        print(f'Procedure {procedure} not exist, try to create it.')

<cx_Oracle.Cursor on <cx_Oracle.Connection to jzsk@10.10.10.155:1521/ORCL>>

<cx_Oracle.Cursor on <cx_Oracle.Connection to jzsk@10.10.10.155:1521/ORCL>>

Directory JD_IMAGE_PATH does not exist, try to create it.


In [8]:
sql = '''select {sequence}.nextval from dual'''

with db.cursor() as cursor:
    cursor.execute(sql.format(sequence=sequence))
    id_num = cursor.fetchone()[0]

<cx_Oracle.Cursor on <cx_Oracle.Connection to jzsk@10.10.10.155:1521/ORCL>>

### 3.2 插入数据

In [9]:
now = datetime.now()
imgPath = './2019-10-28_10-02-01.png'
with open(imgPath, 'rb') as fp:
    imgContent = fp.read()

In [10]:
sql = """insert into {table}
    values (:ID, :DATA_CAMERA_ID, :DATE_TIME, :DATA_IMAGE_PATH, :DATA_INFOR, :DATA_BLOB)"""

with db.cursor() as cursor:
    cursor.execute(
        sql.format(table=table),
        [id_num, 1, now, '00006.png', '类别:船，面积:30，方向：xx，速度：xx', imgContent]
    )
    db.commit()

In [7]:
# clear blob

sql = """update {table} set DATA_BLOB=(:DATA_BLOB) where ID={id_num}"""

with db.cursor() as cursor:
    cursor.execute(
        sql.format(table=table, id_num=2),
        [None]
    )
    db.commit()

## 4. 保存图片到磁盘(服务器)

创建一个存储过程(procedure), 使用时调用即可。输入参数: ID

### 4.1 创建目录

需要注意目录的物理地址应该存在，并且需要管理员赋予当前用户对此文件夹的读写权限。sql 如下

In [ ]:
# sql = """create or replace directory JD_IMAGE_PATH as 'D:\JD_DETECT_IMAGE' """
# with db.cursor() as cursor:
#     cursor.execute(sql)
# sql = """grant read, write on directory JD_IMAGE_PATH to jzsk """
# with db.cursor() as cursor:
#     cursor.execute(sql)

### 4.2 创建 procedure
输入参数为 row 的 id

In [61]:
sql_saveimg2disk = '''
CREATE OR REPLACE
PROCEDURE saveimg2disk(I_ID INTEGER) IS

    file_handle UTL_FILE.FILE_TYPE;
    file_name VARCHAR2(255);
    file_blob BLOB;

    v_buffer       RAW(32767);
    v_buffer_size  BINARY_INTEGER;
    v_amount       BINARY_INTEGER;
    v_offset       NUMBER(38) := 1;
    v_chunksize    INTEGER;

BEGIN

    select DATA_BLOB into file_blob from JD_IMAGE_DETECT where ID=I_ID;
    select DATA_IMAGE_PATH into file_name from JD_IMAGE_DETECT where ID=I_ID;

    -- 大小
    v_chunksize := DBMS_LOB.GETCHUNKSIZE(file_blob);
    IF (v_chunksize < 32767) THEN
         v_buffer_size := v_chunksize;
    ELSE
        v_buffer_size := 32767;
    END IF;
    v_amount := v_buffer_size;

    -- 打开 BLOB
    DBMS_LOB.OPEN(file_blob, DBMS_LOB.LOB_READONLY);

    -- 打开文件
    file_handle := UTL_FILE.FOPEN('JD_IMAGE_PATH', file_name, 'wb', 32767);
    DBMS_OUTPUT.PUT_LINE( '===OPEN OK===' || file_name || '==='|| LENGTH(file_blob));

    -- 循环写入
    WHILE v_amount >= v_buffer_size
    LOOP
        DBMS_LOB.READ(file_blob, v_amount, v_offset, v_buffer);
        v_offset := v_offset + v_amount;
        UTL_FILE.PUT_RAW(file_handle, v_buffer, true);
        UTL_FILE.FFLUSH(file_handle);
    END LOOP;
    UTL_FILE.FFLUSH(file_handle);
    UTL_FILE.FCLOSE(file_handle);

    -- 关闭文件
    DBMS_LOB.CLOSE(file_blob);

EXCEPTION

  WHEN UTL_FILE.INVALID_PATH THEN
    --无效的路径
    DBMS_OUTPUT.PUT_LINE('===INVALID_PATH==='|| I_ID);
    RAISE;

  WHEN UTL_FILE.INVALID_MODE THEN
    --无效的打开模式
    DBMS_OUTPUT.PUT_LINE('===INVALID_MODE==='|| I_ID);
    RAISE;

  WHEN UTL_FILE.INVALID_OPERATION THEN
    --无效的操作，文件打开错误会报这个异常，一般来说都是超长或打开方式byte型和非byte型
    DBMS_OUTPUT.PUT_LINE('===INVALID_OPERATION==='|| I_ID);
    RAISE;

  WHEN UTL_FILE.INVALID_MAXLINESIZE THEN
    --无效的最大长度，RAW最大32676，超过回报这个异常，所以一般要进行循环操作
    DBMS_OUTPUT.PUT_LINE('===INVALID_MAXLINESIZE==='|| I_ID);
    RAISE;

  WHEN UTL_FILE.ACCESS_DENIED THEN
    --拒绝进入指定路径，可能是授权问题
    DBMS_OUTPUT.PUT_LINE('===ACCESS_DENIED==='|| I_ID);
    RAISE;

  WHEN UTL_FILE.INVALID_FILEHANDLE THEN
    --文件处理错误，不常见
    DBMS_OUTPUT.PUT_LINE('===INVALID_FILEHANDLE==='|| I_ID);
    RAISE;

  WHEN UTL_FILE.WRITE_ERROR THEN
    --写入错误，处理该异常最好的方式是将要写入的文件简单化，然后找准错误原因
    DBMS_OUTPUT.PUT_LINE('===WRITE_ERROR==='|| I_ID);
    RAISE;

  WHEN NO_DATA_FOUND THEN
    --SELECT时候未找到数据，不是UTL_FILE的异常
    DBMS_OUTPUT.PUT_LINE('===NO_DATA_FOUND==='|| I_ID);
    UTL_FILE.FCLOSE(file_handle);
    RAISE;

  WHEN OTHERS THEN
    IF UTL_FILE.IS_OPEN(file_handle) THEN
      UTL_FILE.FCLOSE(file_handle);
      RAISE;
    END IF;

END saveimg2disk;
'''

with db.cursor() as cursor:
    cursor.execute(sql_saveimg2disk)


### 4.3 调用 procedure

In [5]:
sql = '''
begin
{procedure}({id_num});
end;
'''

with db.cursor() as cursor:
    cursor.execute(sql.format(procedure=procedure, id_num=2))

In [12]:
# clear blob

sql = """update {table} set DATA_BLOB=(:DATA_BLOB) where ID={id_num}"""

with db.cursor() as cursor:
    cursor.execute(
        sql.format(table=table, id_num=12),
        [None]
    )
    db.commit()

## 5. 查看数据

In [10]:
numRows = 10
sql = f'select * from {table}'

with db.cursor() as cursor:
    cursor.execute(sql)
    for row in cursor.fetchmany(numRows):
        print(row)

<cx_Oracle.Cursor on <cx_Oracle.Connection to jzsk@10.10.10.155:1521/ORCL>>

(1, 5, datetime.datetime(2019, 10, 28, 9, 59, 4), './0001.png', 'label:boat area:30', None)
(2, 5, datetime.datetime(2019, 10, 28, 9, 59, 4), './0001.png', 'label:boat area:30', None)
(12, 5, datetime.datetime(2019, 10, 29, 19, 9, 9), './0001.png', 'label:boat area:30', None)
(13, 5, datetime.datetime(2019, 10, 29, 19, 10, 11), '00005.png', 'label:boat area:30', <cx_Oracle.LOB object at 0x000001E0CEF83378>)
(14, 1, datetime.datetime(2019, 10, 29, 19, 16, 24), '00006.png', 'label:boat area:30', <cx_Oracle.LOB object at 0x000001E0CEF83F08>)


In [ ]:
class oracle:
    cnt = 1
    def __init__(self, user, passwd, host, port, ins, table):
        self.db = cx_Oracle.connect(f'{user}/{passwd}@{host}:{port}/{ins}',
                                    encoding="UTF-8")

    def createTable(self, table):
        cursor = self.db.cursor()
        sql = """CREATE TABLE {table} (
                 DATA_ITEM_ID INT NOT NULL,
                 DATE_TIME VARCHAR(255) NOT NULL,
                 DATA_image_path VARCHAR(255) NOT NULL,
                 DATA_infor VARCHAR(255) NOT NULL
                 )"""
        cursor.execute(sql.format(table))

    def insertOne(self, cameraId, date, imgPath, imgInfo):
        cursor = self.db.cursor()
        try:
            sql = """INSERT INTO {table}
            values (:DATA_ITEM_ID, :DATE_TIME, :DATA_image_path, :DATA_infor)"""
            cursor.execute(
                sql.format(table),
                [cameraId, date, imgPath, imgInfo]
            )

            self.db.commit()
        except cx_Oracle.ProgrammingError:
            print('programming errors')
        except Exception:
            self.db.rollback()
        else:
            print('Insert successfully')

    def __del__(self):
        if hasattr(self, 'db'):
            self.db.close()


test = [1, '2020-10-22 16:03', './imgPath/000001.png', 'label:boat, area:0.3']

testOra = oracle(user, passwd, host, port, ins, table)
